In [4]:
import os
import requests
import re
from googlesearch import search 
from tabula import read_pdf
import urllib.parse
import pandas as pd
from datetime import datetime
import pdfplumber

dirName = "announcements"
dirPath = ".\\" + dirName + "\\"

try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  announcements  already exists


In [72]:
def download_file(url):
    file = requests.get(url, stream=True)
    headers = file.headers['content-disposition']
    filename = re.findall("filename=\"(.+)\"", headers)[0]
    location = os.path.relpath(dirPath + filename)
    open(location, 'wb').write(file.content)
    del file

In [70]:
query = "\"S&P 500 INDEX –\" \"Set to Join\" site:spindices.com filetype:pdf"
results = search(query, tld='com', lang='en', num=10, start=0, stop=None, pause=2.0)

In [74]:
for result in results:
    download_file(result)
    print(result)

https://us.spindices.com/documents/indexnews/announcements/20200122-1077359/1077359_5payc.pdf?force_download=true
https://us.spindices.com/documents/indexnews/announcements/20190709-957359/957359_hat5mobile.pdf?force_download=true
https://us.spindices.com/documents/indexnews/announcements/20200227-1098962/1098962_Final5Ingersol4Cimar.pdf?force_download=true
https://www.spindices.com/documents/indexnews/announcements/20190314-892576/892576_21foxspindis100adbe.pdf
https://www.spindices.com/documents/indexnews/announcements/20191127-1041559/1041559_BerkleyRJoe546.pdf
https://www.spindices.com/documents/indexnews/announcements/20190801-969367/969367_1leidos546foot6fizz6.pdf
https://us.spindices.com/documents/indexnews/announcements/20190624-948566/948566_mktxaaxnmlab546usna6xncr6.pdf?force_download=true
https://www.spindices.com/documents/indexnews/announcements/20190326-898958/898958_1dowspin100trex400.pdf
https://www.spindices.com/documents/indexnews/announcements/20190926-1004359/100435

ConnectionError: HTTPConnectionPool(host='stg.spindices.com', port=80): Max retries exceeded with url: /documents/indexnews/announcements/20150706-204357/204357_dollarjackcontact546.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000021D19A414C8>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [299]:
columns = ['type', 'name', 'ticker', 'announcement_date', 'implementation_date']
df = pd.DataFrame(columns=columns)

In [298]:
def find_datetime_from_text(string):
    search_results = re.search('[A-z]+ \d\d?(\W+)\d{4}', string)
    if search_results is None:
        print('No date found in ' + string)
        return None
    matched_string = search_results.group(0)
    splitted_date_string = re.split("\W+", matched_string)
    date_string = ' '.join(splitted_date_string)
    return datetime.strptime(date_string, '%B %d %Y')

In [178]:
def get_ticker_by_name(stock_name):
    '''
    Takes in a stock_name in string,
    Returns the ticker in string
    '''
    encoded_stock_name = urllib.parse.quote(stock_name)
    url = "https://s.yimg.com/aq/autoc?query=" + encoded_stock_name + "&lang=en-US"
    response = requests.get(url)
    json_response = response.json()
    results = json_response['ResultSet']['Result']
    if len(results) == 0:
        print('No ticker found.')
        ticker = None
    else:
        ticker = results[0]['symbol']
    response.close()
    return ticker

In [350]:
def get_ticker(stock_name):
#     for page in pages:
#         text = page.extract_text()
    text = "Arista Networks Inc. (NYSE: ANET) will replace GGP Inc. (NYSE: GGP) in the S&P 500 effective prior to the open of trading on Tuesday, August 28. Brookfield Property Partners LP (NASD: BPY) is acquiring GGP in a deal expected to be completed soon pending final conditions."
    pattern = stock_name + ".+?\([A-Z]+:\W+([A-Z]+)\)"
    matches = re.search(pattern, text)
    return matches

In [351]:
stock_name = 'Arista Networks'
ticker = get_ticker(stock_name)

In [353]:
ticker.group(1)

'ANET'

In [307]:
stock_name = "GGP"
encoded_stock_name = urllib.parse.quote(stock_name)
url = "https://s.yimg.com/aq/autoc?query=" + encoded_stock_name + "&lang=en-US"
response = requests.get(url)
json_response = response.json()
results = json_response['ResultSet']['Result']
if len(results) == 0:
    print('No ticker found.')
    ticker = None
else:
    ticker = results[0]['symbol']
response.close()

In [308]:
json_response

{'ResultSet': {'Query': 'GGP',
  'Result': [{'symbol': 'GGPCX',
    'name': 'American Funds Global Growth Portfolio Class C',
    'exch': 'NAS',
    'type': 'M',
    'exchDisp': 'NASDAQ',
    'typeDisp': 'Fund'},
   {'symbol': 'GGPXF',
    'name': 'G-Resources Group Limited',
    'exch': 'PNK',
    'type': 'S',
    'exchDisp': 'OTC Markets',
    'typeDisp': 'Equity'},
   {'symbol': 'GGPPX',
    'name': 'Goldman Sachs Growth Strategy Portfolio Class P',
    'exch': 'NAS',
    'type': 'M',
    'exchDisp': 'NASDAQ',
    'typeDisp': 'Fund'},
   {'symbol': 'GGP.L',
    'name': 'Greatland Gold plc',
    'exch': 'YHD',
    'type': 'S',
    'exchDisp': 'Industry',
    'typeDisp': 'Equity'},
   {'symbol': 'GGPL.BO',
    'name': 'Gala Global Products Limited',
    'exch': 'BSE',
    'type': 'S',
    'exchDisp': 'Bombay',
    'typeDisp': 'Equity'},
   {'symbol': 'GGP.MI',
    'name': 'Gruppo Green Power S.p.A.',
    'exch': 'YHD',
    'type': 'S',
    'exchDisp': 'Industry',
    'typeDisp': 'Equi

In [306]:
json_response

{'ResultSet': {'Query': 'RHT',
  'Result': [{'symbol': 'RHT.V',
    'name': 'Reliq Health Technologies Inc.',
    'exch': 'VAN',
    'type': 'S',
    'exchDisp': 'CDNX',
    'typeDisp': 'Equity'},
   {'symbol': 'RHT.AX',
    'name': 'Resonance Health Limited',
    'exch': 'ASX',
    'type': 'S',
    'exchDisp': 'Australian',
    'typeDisp': 'Equity'},
   {'symbol': 'RHTLN0000.CM',
    'name': 'FORTRESS RESORTS',
    'exch': 'CSE',
    'type': 'S',
    'exchDisp': 'Colombo Stock Exchange',
    'typeDisp': 'Equity'}]}}

In [304]:
json_response

{'ResultSet': {'Query': 'Carrier Global',
  'Result': [{'symbol': 'CARR-WI',
    'name': 'Carrier Global Corporation',
    'exch': 'NYQ',
    'type': 'S',
    'exchDisp': 'NYSE',
    'typeDisp': 'Equity'},
   {'symbol': 'CARR',
    'name': 'Carrier Global Corporation',
    'exch': 'NYQ',
    'type': 'S',
    'exchDisp': 'NYSE',
    'typeDisp': 'Equity'},
   {'symbol': '4PN.F',
    'name': 'Carrier Global Corporation',
    'exch': 'FRA',
    'type': 'S',
    'exchDisp': 'Frankfurt',
    'typeDisp': 'Equity'},
   {'symbol': 'CARR.MX',
    'name': 'Carrier Global Corporation',
    'exch': 'MEX',
    'type': 'S',
    'exchDisp': 'Mexico',
    'typeDisp': 'Equity'}]}}

In [300]:

for filename in os.listdir(dirPath):
    pdf = pdfplumber.open(dirPath + filename)
    announcement_date = find_datetime_from_text(pdf.pages[0].extract_text())
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            if any("S&P 500 INDEX" in str(item) for item in table[0]):
                print(table)
                implementation_date = find_datetime_from_text(table[0][0])
                last_change_type = ""
                for i in range(2, len(table)):
                    
                    change_type = table[i][0]
                    if change_type == "" or change_type == None:
                        change_type = last_change_type
                    else:
                        last_change_type = change_type
                    
                    name = table[i][1].replace('\n', '')
                    ticker = get_ticker_by_name(name)
                    row = [{'type': change_type, 'name': name, 'ticker': ticker, 
                            'implementation_date': implementation_date, 'announcement_date': announcement_date}]
                    df = df.append(pd.DataFrame(row), ignore_index=True)

[['S&P 500 INDEX – September 26, 2019', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'NVR', 'Consumer Discretionary', 'Homebuilding'], ['DELETED', 'Jefferies \nFinancial Group', 'Financials', 'Multi-Sector Holdings']]
[['S&P 500 INDEX – October 3, 2019', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Las Vegas \nSands', 'Consumer Discretionary', 'Casinos & Gaming'], ['DELETED', 'Nektar \nTherapeutics', 'Health Care', 'Pharmaceuticals']]
[['S&P 500 INDEX – December 5, 2019', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'W.R. Berkley', 'Financials', 'Property & Casualty \nInsurance'], ['DELETED', 'Viacom', 'Communication Services', 'Movies & Entertainment']]
[['S&P 500 INDEX – December 9, 2019', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Old Dominion \nFreight Line', 'Industrials', 'Trucking'], ['DELETED'

[['S&P 500 INDEX – April 22, 2016', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Global Payments', 'Information Technology', 'Data Processing & Outsourced \nServices'], ['DELETED', 'GameStop', 'Consumer Discretionary', 'Computer & Electronic Retail']]
[['S&P 500 INDEX – MAY 12, 2016', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Alaska Air', 'Industrials', 'Airlines'], ['DELETED', 'SanDisk', 'Information Technology', 'Technology Hardware, \nStorage & Peripherals']]
No ticker found.
[['S&P 500 INDEX – MAY 17, 2016', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Digital Realty \nTrust', 'Financials', 'Specialized REITs'], ['DELETED', 'Time Warner \nCable', 'Consumer Discretionary', 'Cable & Satellite']]
No ticker found.
[['S&P 500 INDEX – MAY 20, 2016', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'LKQ', 

[['S&P 500 INDEX – MAY 31, 2018', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'ABIOMED', 'Health Care', 'Health Care Equipment'], ['DELETED', 'Wyndham \nWorldwide', 'Consumer Discretionary', 'Hotels, Resorts & Cruise \nLines']]
[['S&P 500 INDEX – JUNE 7, 2018', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC \nSECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Twitter', 'Information Technology', 'Internet Software & Services'], ['DELETED', 'Monsanto', 'Materials', 'Fertilizers & Agricultural \nChemicals']]
No ticker found.
[['S&P 500 INDEX – June 18, 2018', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'HollyFrontier', 'Energy', 'Oil & Gas Refining & \nMarketing'], ['', 'Broadridge \nFinancial \nSolutions', 'Information Technology', 'Data Processing & \nOutsourced Services'], ['DELETED', 'Acuity Brands', 'Industrials', 'Electrical Components \n& Equipment'], ['', 'Range \nResources', 'Energy', 'Oil & G

In [301]:
df

,type,name,ticker,announcement_date,implementation_date
0,ADDED,NVR,NVR,2019-09-20,2019-09-26
1,DELETED,Jefferies Financial Group,JEF,2019-09-20,2019-09-26
2,ADDED,Las Vegas Sands,LVS,2019-09-26,2019-10-03
3,DELETED,Nektar Therapeutics,NKTR,2019-09-26,2019-10-03
4,ADDED,W.R. Berkley,WRB-PF,2019-11-27,2019-12-05
...,...,...,...,...,...
185,,SAIC,SAIC,2013-09-11,2013-09-20
186,ADDED,Arista Networks,ANET,2018-08-23,2018-08-28
187,DELETED,GGP,GGPCX,2018-08-23,2018-08-28
188,ADDED,AbbVie,ABBV,2012-12-20,2012-12-31


In [302]:
df.to_excel('output.xlsx')